In [1]:
#pip install --upgrade simplevec

In [2]:
pip show simplevec

Name: simplevec
Version: 0.0.2
Summary: SimpleVec: Elementary Examples of Vector Modeling
Home-page: https://github.com/jrolf/simplevec
Author: James A. Rolfsen
Author-email: james.rolfsen@think.dev
License: UNKNOWN
Location: /Users/jar/opt/anaconda3/lib/python3.8/site-packages
Requires: statsmodels, gensim, sympy, scipy, scikit-learn, xgboost, pandas, numpy
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [3]:
from simplevec import * 

In [20]:
#https://stackoverflow.com/questions/57148357/attributeerror-module-gensim-utils-has-no-attribute-smart-open
#pip install gensim==3.4.0

from gensim.models import Word2Vec as word2vec 

In [4]:
#############################################################################
############################################################################# 

In [5]:
#############################################################################
#############################################################################

In [6]:
vec1 = user2vec['U0001']
vec2 = item2vec['I0001'] 

vec1 = np.array([vec1])
vec2 = np.array([vec2]) 

VecDist(vec1,vec2)  

array([10.618184606768507], dtype=object)

In [7]:
print(vec1.shape)
print(vec2.shape) 

(1, 6)
(1, 6)


In [8]:
#############################################################################
#############################################################################

In [9]:
#############################################################################
#############################################################################

In [10]:
RevDF['SALE'].mean()

0.44360787660793755

In [11]:
RevDF['SALE'].sum() / RevDF['UID'].nunique() 

14.553

In [12]:
#RevDF.head(10)

In [13]:
RevDF.columns

Index(['UID', 'IID', 'DIST', 'DPER', 'AFFIN', 'RATING', 'SALE'], dtype='object')

In [14]:
#cols = ['UID', 'IID','RATING']
#RevDF2 = RevDF[cols].copy() 

In [15]:
#RevDF2.head(10) 

In [16]:
Sales = RevDF[RevDF['SALE']==1].copy() 
Sales = RevDF.copy() 

In [17]:
Sales

,UID,IID,DIST,DPER,AFFIN,RATING,SALE
0,U0001,I0018,13.164091,0.106325,0.798655,8,1
1,U0001,I0066,11.805605,0.065234,0.873788,9,1
2,U0001,I0128,24.929997,0.860052,0.019585,1,0
3,U0001,I0131,15.593110,0.219235,0.609594,6,1
4,U0001,I0142,28.310396,0.965463,0.001193,1,0
...,...,...,...,...,...,...,...
32801,U1000,I0426,16.315121,0.263222,0.542842,6,1
32802,U1000,I0440,18.580588,0.421978,0.334109,4,0
32803,U1000,I0441,13.926884,0.135498,0.747364,8,1
32804,U1000,I0444,17.455853,0.339674,0.436031,5,0


In [18]:
# ['UID' ,'IID' ,'SALE']
# ['ENT1','ENT2','SCORE'] 

In [19]:
#############################################################################
#############################################################################
#############################################################################

In [ ]:
Notes = '''

model = SimpleVecModel() 

# Given: A Matrix of ['UID','IID'] pairs, "relations":

model.fit(relations,vec_len=30)  

vec = model.vec[UID] 

cos_sim = model.cos_sim(A,B) 


'''

In [65]:

W2V = word2vec() 
# Cosine Similarity of one Vector to a Matrix: 
def cosine_sim_mat(vec_target,vec_matrix):
    return W2V.wv.cosine_similarities(vec_target,vec_matrix) 
  
def cosine_sim_vec(x, y):
    return np.dot(x, y) / (np.sqrt(np.dot(x, x)) * np.sqrt(np.dot(y, y)))

def normalize(vec1):
    m = vec1.mean()
    vec2 = vec1-m
    return vec2/(vec2.std()) 

def getLinSim(vec_target,vec_matrix,exp=1.0):
    return 1-(np.abs(M-np.array([t]))**exp).mean(1) 

#--------------------------------------------------------------------

class SimpleVecModel:
    
    def __init__(self):
        self.x = 1
        
    def fit(self,relations,add_prefix=True):
        rels = np.array(relations).astype(str)
        # Shuffles the training data. (Good practice for large data sets.) 
        for _ in range(7): np.random.shuffle(rels) 
        
        self.ent2id = dict()
        self.id2ent = dict()
        
        ents1 = np.array(sorted(set(rels[:,0]))) 
        for ent in ents1:
            if add_prefix: ent_id = 'ENT1_'+str(ent) 
            else:          ent_id = str(ent)
            self.ent2id[ent] = ent_id
            self.id2ent[ent_id] = ent
        ids1 = np.array([self.ent2id[ent] for ent in ents1])
        
        ents2 = np.array(sorted(set(rels[:,1]))) 
        for ent in ents2:
            if add_prefix: ent_id = 'ENT2_'+str(ent) 
            else:          ent_id = str(ent)
            self.ent2id[ent] = ent_id
            self.id2ent[ent_id] = ent
        ids2 = np.array([self.ent2id[ent] for ent in ents2])
        
        rels2 = []
        for r in rels:
            ent1,ent2 = tuple(r) 
            id1,id2 = self.ent2id[ent1],self.ent2id[ent2] 
            rels2.append([id1,id2]) 
        
        self.w2v = word2vec(
            rels2,
            vector_size=10,
            epochs=40,
            alpha=0.0123,
            min_count=3,  
            negative=25,
            batch_words=100000,
            workers=9    
        );
        
        self.id2vec = dict() 
        for id1 in ids1:
            try: vec = self.w2v.wv[id1]
            except: continue
            self.id2vec[id1] = vec
        for id2 in ids2:
            try: vec = self.w2v.wv[id2]
            except: continue
            self.id2vec[id2] = vec
            
        self.all_found_ids  = np.array(sorted(self.id2vec)) 
        self.all_found_ents = np.array([self.id2ent[fid] for fid in self.all_found_ids]) 
        self.all_found_vecs = np.array([self.id2vec[fid] for fid in self.all_found_ids])
        
        print('Done fitting embedding model.') 
        
    def ent2vec(self,ent):
        try:
            ent_id = self.ent2id[str(ent)]
            vec = self.id2vec[ent_id] 
            return normalize(vec) 
        except:
            return np.array([]) 
        
    def similar(self,tgt_ent,other_ents=[],n_similar=25): 
        if len(other_ents)>0:
            self.found_ent_set = set(self.all_found_ents) 
            found_ents = np.array([a for a in other_ents if ent in self.found_ent_set]) 
            M = np.array(sorted([self.ent2vec(ent) for ent in found_ents]))  
        else: 
            M = self.all_found_vecs
            found_ents = np.array(self.all_found_ents) 
        tgt_vec = self.ent2vec(tgt_ent) 
        sims = cosine_sim_mat(tgt_vec,M) 
        order = np.argsort(sims)[::-1] 
        top_results = found_ents[order][:n_similar+2]  
        return [a for a in top_results if a!=tgt_ent][:n_similar] 
            


In [66]:
#############################################################################
#############################################################################

In [72]:
relations = RevDF[RevDF['SALE']==1][['UID','IID']].values
print(relations[:10]) 

[['U0001' 'I0018']
 ['U0001' 'I0066']
 ['U0001' 'I0131']
 ['U0001' 'I0234']
 ['U0001' 'I0454']
 ['U0002' 'I0047']
 ['U0002' 'I0130']
 ['U0002' 'I0135']
 ['U0002' 'I0193']
 ['U0002' 'I0195']]


In [67]:
model = SimpleVecModel() 
model.fit(relations) 

Done fitting embedding model.


In [68]:
relations

array([['U0001', 'I0018'],
       ['U0001', 'I0066'],
       ['U0001', 'I0131'],
       ...,
       ['U1000', 'I0426'],
       ['U1000', 'I0441'],
       ['U1000', 'I0486']], dtype=object)

In [76]:
ent1 = 'U0001'
vec1 = model.ent2vec(ent1) 
vec1

array([ 0.4373381 , -0.41223752,  0.14855556, -0.03600831,  0.45119897,
        0.3741549 ,  0.6578923 ,  0.751126  , -0.7877293 , -0.4883157 ],
      dtype=float32)

In [77]:
matches = model.similar(ent1) 

In [78]:
matches[:5] 

['U0183', 'U0262', 'U0611', 'U0900', 'U0541']

In [79]:
ent2 = 'U0183'
vec2 = model.ent2vec(ent2)  
vec2

array([ 0.43359825, -0.39493355,  0.21564676, -0.05154037,  0.45017964,
        0.4248314 ,  0.7092857 ,  0.801722  , -0.83830285, -0.54948926],
      dtype=float32)

In [87]:
ent_id = model.ent2id[ent1] 
results = model.w2v.predict_output_word([ent_id],10)  
id_matches = [a[0] for a in results] 
match_vecs = np.array([model.id2vec[ent_id] for ent_id in id_matches]) 

In [89]:
ave_vec = normalize(match_vecs.sum(0)) 

In [90]:
ave_vec

array([ 0.58566844, -0.93374795,  0.229962  , -0.50346017,  0.7876514 ,
        0.75232744,  0.96299195,  1.1507213 , -1.9872206 , -1.0448939 ],
      dtype=float32)

In [ ]:
#############################################################################
#############################################################################

In [95]:
### EVALUATE THE EMBEDDINGS BY CALCULATING THE LATENT DISTANCE OF THE TRUE EMBEDDINGS 
### AND COMPARING TO THE COSINE SIM OF THE INFERRED/MODELED EMBEDDINGS.

TDF = RevDF[['UID','IID','DIST']].copy() 

results = []
for r in TDF.values:
    ent1,ent2,dist = tuple(r) 
    
    vec1 = model.ent2vec(ent1)
    vec2 = model.ent2vec(ent2) 
    
    if len(vec1)<1 or len(vec2)<1: continue 
    cos_sim = cosine_sim_vec(vec1,vec2)
    results.append([ent1,ent2,dist,cos_sim]) 
    
RDF = pd.DataFrame(results)
RDF.columns = ['UID','IID','DIST','COS_SIM'] 
    
coverage = len(RDF)/len(TDF) 
print('Evaluation Coverage:',round(coverage,6))

Evaluation Coverage: 0.99241


In [97]:
RDF['COS_SIM'].mean() 

0.97363615

In [98]:
vec1

array([ 0.45372668, -0.36753684,  0.16939509, -0.11888527,  0.5808792 ,
        0.44169354,  0.67561275,  0.8302817 , -1.0012454 , -0.4402082 ],
      dtype=float32)

In [99]:
vec2

array([ 0.40636188, -0.33068043,  0.36374336, -0.25951698,  0.52015936,
        0.5081867 ,  0.71339184,  0.8053956 , -1.0278873 , -0.53701967],
      dtype=float32)